In [38]:
import cv2
import pytesseract
import easyocr
import numpy as np
import re

# Manually define the known brands
known_brands = [
    "Lays", "5 Star", "Kitkat", "All Out", "Odonil", "Aashirwaad", 
    "Amul", "Beardo", "Bisleri", "Boost", "Bournvita", "Bru", 
    "Fortune", "Dettol", "Harpic", "Horlicks", "Parle-G", 
    "Kinley", "KrackJack", "Limea", "Maggie", "Yippee", 
    "Marie Gold", "Mirinda", "Munch", "Nestle", "Rin", 
    "Santoor", "Vim", "Sprite", "Stayfree", "Whisper", 
    "Thumbs-Up", "Wildstone"
]

# Image Preprocessing Functions
def adjust_contrast(image):
    return cv2.equalizeHist(image)

def denoise_image(image):
    return cv2.fastNlMeansDenoising(image, None, 30, 7, 21)

def sharpen_image(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

def adaptive_threshold(image):
    return cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

def deskew_image(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# Multiple passes of OCR with different preprocessing techniques
def multiple_pass_ocr(image):
    processed_images = [sharpen_image(image), adaptive_threshold(image), denoise_image(image)]
    all_extracted_text = []
    
    for img in processed_images:
        extracted_text = pytesseract.image_to_string(img, config=r'--oem 3 --psm 6')
        all_extracted_text.append(extracted_text)
    
    return ' '.join(all_extracted_text)

# Regex to extract specific details like brand, pack size, product name, etc.
def extract_details_using_regex(text, known_brands):
    text = text.lower()
    
    # Initialize default values
    pack_size = "Pack size not found"
    brand_name = "Brand name not found"
    product_name = "Product name not found"
    
    # Attempt to extract the pack size using regex
    pack_size_matches = re.findall(r'net\s+wt:\s*(\d+\.?\d*)\s*(oz|g|kg|ml|l)', text)

    # Check for brand name in text
    for brand in known_brands:
        if brand.lower() in text:
            brand_name = brand  # Keep the original case
            break

    # Extract product name
    if brand_name != "Brand name not found":
        # Use regex to capture product name after brand name
        product_name_matches = re.findall(r'{}(?:.*?)(?=net\s+wt:)'.format(re.escape(brand_name.lower())), text)
        if product_name_matches:
            product_name = product_name_matches[0].strip()

    # Reverting to previous pack size extraction logic
    if pack_size_matches:
        pack_size = f"{pack_size_matches[0][0]} {pack_size_matches[0][1].upper()}"
    else:
        # Additional regex to capture other pack size formats
        pack_size_alt_matches = re.findall(r'(\d+\.?\d*)\s*(oz|g|kg|ml|l)', text)
        if pack_size_alt_matches:
            pack_size = f"{pack_size_alt_matches[0][0]} {pack_size_alt_matches[0][1].upper()}"

    return {
        "Pack Size": pack_size,
        "Brand Name": brand_name,
        "Product Name": product_name
    }

# EasyOCR extraction
def extract_text_easyocr(image_path):
    reader = easyocr.Reader(['en'])
    extracted_texts = reader.readtext(image_path, detail=0)
    return ' '.join(extracted_texts)

# Hybrid OCR combining Tesseract and EasyOCR
def hybrid_ocr(image, image_path):
    tesseract_text = multiple_pass_ocr(image)
    easyocr_text = extract_text_easyocr(image_path)
    combined_text = ' '.join([tesseract_text, easyocr_text])
    return combined_text

if __name__ == "__main__":
    image_path = 'Lays.jpeg'  # Update this with the path to your image

    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contrast_image = adjust_contrast(gray_image)
    deskewed_image = deskew_image(contrast_image)

    combined_text = hybrid_ocr(deskewed_image, image_path)

    details = extract_details_using_regex(combined_text, known_brands)

    print("--- Extracted Details ---")
    print(f"Brand Name: {details['Brand Name']}")
    print(f"Pack Size: {details['Pack Size']}")
    print(f"Product Name: {details['Product Name']}")


[2024-10-11 17:20:15,531] [ WARNING] easyocr.py:80 - Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


--- Extracted Details ---
Brand Name: Lays
Pack Size: 10 OZ
Product Name: lays bran the classic potato chips
